# Prediction model for diabetes dataset

This notebook assumes the use of the cleaned dataset (notebook EDA.ipynb).

In [3]:
#imports:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
# from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns



In [4]:
data = pd.read_csv('diabetes_cleaned.csv')
#feature to predict:
target_feature = 'CLASS' #the column to predict

# Dataset cross validation - splittng

In [5]:
# Splitting the dataset into training and testing sets
X = data.drop(columns=[target_feature])  # Features to use in the model
y = data[target_feature]  # Target feature of the dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

Training set size: 795 samples
Testing set size: 199 samples
